#### git.ignore
#### create .gitignore_global
echo '.DS_Store' >> ~/.gitignore_global


#### open .gitignore_global
code ~/.gitignore_global

#### code withing .gitignore_global
env.py
.DS_Store
.ipynb_checkpoints/
__pycache__
.vscode/

#### code to exclude git from tracking .gitignore_global
git config --global core.excludesfile ~/.gitignore_global



In [1]:
# Establish connection to Codeup database
from env import host, user, password
import os
import acquire
import prep

# data prep
import prep

# data manipulation/
import pandas as pd
import numpy as np

# splitting and imputing functions
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# modeling
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# acquire

# This function will establish a callable connection to the Codeup db
def get_connection(db, user=user, host=host, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'


In [3]:
# This function will read the telco_churn data, from the Codeup database, to a dataframe.
def telco_data():
    sql_query = '''
        select * from customers
                join contract_types using (contract_type_id)
                join internet_service_types using (internet_service_type_id)
                join payment_types using (payment_type_id)
        '''
    df = pd.read_sql(sql_query, get_connection('telco_churn'))
    return df

In [4]:
# this function will call the telco_churn data from the Codeup db to a csv file
def get_telco_data():
    if os.path.isfile('telco.csv'):
        df = pd.read_csv('telco.csv', index_col=0)
    else:
        df = telco_data()
        df.to_csv('telco.csv')
    return df

In [5]:
df = acquire.get_telco_data()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

In [4]:
# create function to clean data
def clean_data(df):
    '''This function will take in a dataframe and search for 
'payment_type_id', 'internet_service_type_id',  'contract_type_id'.
If the specified columns are in the dataset,  drop_cols in the if 
statement will be executed.
If the specified columns are not in the dataset, drop_cols in the 
else statement will execute.
For columns in dummy_col, dummy columns will be created
without dropping the first column.
For columns in dummies, dummy columns will be created and the 
first columns will be dropped
Dummy columns from dummy_col will first be concated to the 
original dataframe, then dummy columns from dummies will be 
concated to the dataframe created by dummy_col and the original 
dataframe.
Column names are then formatted to be lowercase, and spaces in 
column names are changed to underscores.
Lastly, the appropriate drop_cols is executed, and specified 
columns are renamed'''
    dummy_cols = ['multiple_lines']
    dummies = ['gender'
               , 'partner'
               , 'dependents']
    if 'payment_type_id' or 'internet_service_type_id' or 'contract_type_id' in df.columns:
        drop_cols = ['payment_type_id'
                     , 'internet_service_type_id'
                     , 'contract_type_id'
                     , 'multiple_lines'
                     , 'contract_type'
                     , 'internet_service_type'
                     , 'payment_type'
                     , 'online_security'
                     , 'online_backup'
                     , 'device_protection'
                     , 'tech_support'
                     , 'streaming_tv'
                     , 'streaming_movies'
                     , 'gender'
                     , 'partner'
                     , 'dependents'
                     , 'phone_service'
                     , 'paperless_billing'
                     , 'device_protection_no_internet_service'
                     , 'online_backup_no_internet_service'
                     , 'online_security_no_internet_service'  
                     , 'tech_support_no_internet_service' 
                     , 'streaming_tv_no_internet_service'
                     , 'streaming_movies_no_internet_service'] 
        dummy = pd.concat([df,
            pd.get_dummies(df[dummy_cols])], axis=1)
        dummy_1 = pd.concat([dummy,
            pd.get_dummies((df[dummies]), drop_first=True)], axis=1)
        dummy_1.columns = dummy_1.columns.str.lower().str.replace(' ', '_')
    else:
        drop_cols = ['multiple_lines'
                     , 'contract_type' 
                     , 'internet_service_type' 
                     , 'payment_type'
                     , 'online_security'
                     , 'online_backup'
                     , 'device_protection'
                     , 'tech_support'
                     , 'streaming_tv'
                     , 'streaming_movies'
                     , 'gender'
                     , 'partner'
                     , 'dependents'
                     , 'phone_service'
                     , 'paperless_billing'
                     , 'device_protection_no_internet_service'
                     , 'online_backup_no_internet_service'
                     , 'online_security_no_internet_service'  
                     , 'tech_support_no_internet_service' 
                     , 'streaming_tv_no_internet_service'
                     , 'streaming_movies_no_internet_service']
        dummy = pd.concat([df,
            pd.get_dummies(df[dummies])], axis=1)
        dummy_1 = pd.concat([dummy,
            pd.get_dummies((df[dummies]), drop_first=True)], axis=1)
        dummy_1.columns = dummy_1.columns.str.lower().str.replace(' ', '_')
    return dummy_1.drop(columns= drop_cols).rename(columns= {'gender_male': 'is_male'
                                                             , 'partner_yes': 'has_partner'
                                                             , 'dependents_yes': 'has_dependents'
                                                             , 'phone_service_yes': 'has_phone_serv'
                                                             , 'paperless_billing_yes': 'has_paperless_bill'
                                                             , 'multiple_lines_no': 'single_line'
                                                             , 'multiple_lines_no_phone_service': 'no_phone_service'
                                                             , 'multiple_lines_yes': 'has_mult_lines'
                                                             , 'contract_type_month-to-month': 'm_to_m_contract'
                                                             , 'contract_type_one_year': 'one_yr_contract'
                                                             , 'contract_type_two_year': 'two_yr_contract'
                                                             , 'contract_type_two_year': 'two_yr_contract'
                                                             , 'internet_service_type_dsl': 'dsl'
                                                             , 'internet_service_type_fiber_optic': 'fiber_optic'
                                                             , 'internet_service_type_none': 'no_internet'
                                                             , 'payment_type_bank_transfer_(automatic)': 'bank_transfer'
                                                             , 'payment_type_credit_card_(automatic)': 'credit_card'
                                                             , 'payment_type_electronic_check': 'e_check'
                                                             , 'payment_type_mailed_check': 'mailed_check'
                                                             , 'online_security_no': 'no_online_security'
                                                             , 'online_security_yes': 'has_online_secur'
                                                             , 'online_backup_no': 'no_online_backup'
                                                             , 'online_backup_yes': 'has_online_backup'
                                                             , 'device_protection_no': 'no_device_protect'
                                                             , 'device_protection_yes': 'has_device_protect'
                                                             , 'tech_support_no': 'no_tech_support'
                                                             , 'tech_support_yes': 'has_tech_support'
                                                             , 'streaming_tv_no': 'not_streaming_tv'
                                                             , 'streaming_tv_yes': 'is_streaming_tv'
                                                             , 'streaming_movies_no': 'not_streaming_movies'
                                                             , 'streaming_movies_yes': 'is_streaming_movies'
                                                             })
# validate data has been cleaned
df = clean_data(df)
df.head()

,customer_id,senior_citizen,tenure,monthly_charges,total_charges,churn,single_line,no_phone_service,has_mult_lines,m_to_m_contract,...,has_tech_support,not_streaming_tv,is_streaming_tv,not_streaming_movies,is_streaming_movies,is_male,has_partner,has_dependents,has_phone_serv,has_paperless_bill
0,0002-ORFBO,0,9,65.6,593.3,No,1,0,0,0,...,1,0,1,1,0,0,1,1,1,1
1,0003-MKNFE,0,9,59.9,542.4,No,0,0,1,1,...,0,1,0,0,1,1,0,0,1,0
2,0004-TLHLJ,0,4,73.9,280.85,Yes,1,0,0,1,...,0,1,0,1,0,1,0,0,1,1
3,0011-IGKFF,1,13,98.0,1237.85,Yes,1,0,0,1,...,0,0,1,0,1,1,1,0,1,1
4,0013-EXCHZ,1,3,83.9,267.4,Yes,1,0,0,1,...,1,0,1,1,0,0,1,0,1,1


In [9]:
# split data
train, test = train_test_split(df
                               , test_size=.2
                               , random_state=123
                               , stratify=df.has_churn)
train, validate = train_test_split(train
                                  , test_size=.3
                                  , random_state=123
                                  , stratify=train.has_churn)

# validate split
print(f'train -> {train.shape}')
print(f'validate -> {validate.shape}')
print(f'test -> {test.shape}')

AttributeError: 'DataFrame' object has no attribute 'has_churn'

In [ ]:
# create train, validate, test function
def split_data(df):
    train, test = train_test_split(df
                               , test_size=.2
                               , random_state=123
                               , stratify=df.has_churn)
    train, validate = train_test_split(train
                                , test_size=.3
                                , random_state=123
                                , stratify=train.has_churn)
    return train, validate, test

# validate function split data
train, validate, test = split_data(df)

print(f'train -> {train.shape}')
print(f'validate -> {validate.shape}')
print(f'test -> {test.shape}')

In [18]:
# create a function that cleans and preps data
def prep_telco(df):
    df = clean_data(df)
    train, validate, test = split_data(df)
    return train, validate, test

# validate function works
train, validate, test = prep.prep_telco(df)

print(f'train -> {train.shape}')
print(f'validate -> {validate.shape}')
print(f'test -> {test.shape}')

train -> (3943, 36)
validate -> (1691, 36)
test -> (1409, 36)


In [19]:
train

,customer_id,senior_citizen,tenure,monthly_charges,total_charges,churn,single_line,no_phone_service,has_mult_lines,m_to_m_contract,...,has_tech_support,not_streaming_tv,is_streaming_tv,not_streaming_movies,is_streaming_movies,is_male,has_partner,has_dependents,has_phone_serv,has_paperless_bill
5310,7503-MIOGA,1,72,89.85,6697.35,No,0,0,1,0,...,1,0,1,0,1,0,1,0,1,1
3790,5329-KRDTM,1,72,77.35,5396.25,No,0,0,1,0,...,1,0,1,1,0,1,1,0,1,0
4398,6199-IWKGC,1,46,100.25,4753.85,No,0,0,1,0,...,1,0,1,0,1,0,1,0,1,0
2635,3748-FVMZZ,0,4,40.05,162.45,No,0,1,0,1,...,1,1,0,0,1,1,0,0,0,1
2986,4280-DLSHD,0,8,54.75,445.85,No,1,0,0,1,...,0,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6230,8809-RIHDD,0,72,103.40,7372.65,Yes,0,0,1,0,...,0,0,1,0,1,1,1,1,1,1
356,0523-VNGTF,1,52,50.50,2566.3,No,0,1,0,1,...,1,0,1,1,0,0,0,0,0,1
2128,3058-HJCUY,0,41,102.60,4213.35,Yes,0,0,1,0,...,1,0,1,0,1,1,1,1,1,1
3586,5081-NWSUP,0,10,64.90,685.55,No,1,0,0,0,...,1,1,0,0,1,0,0,0,1,0


In [23]:
# nothing to see folks

# no_online_security
plt.subplot(428)
plt.title('churn x no_online_security')
sns.boxplot(x="no_online_security", y="churn", data=train)

# credit_card
plt.subplot(422)
plt.title('churn x credit_card')
sns.boxplot(x="credit_card", y="churn", data=train)

# e_check
plt.subplot(424)
plt.title('churn x e_check')
sns.boxplot(x="e_check", y="churn", data=train)

# mailed_check
plt.subplot(426)
plt.title('churn x mailed_check')
sns.boxplot(x="mailed_check", y="churn", data=train)

# has_online_secur
plt.subplot(422)
plt.title('churn x has_online_secur')
sns.boxplot(x="has_online_secur", y="churn", data=train)

# has_tech_support
plt.subplot(424)
plt.title('churn x has_tech_support')
sns.boxplot(x="has_tech_support", y="churn", data=train)

# has_online_backup
plt.subplot(426)
plt.title('churn x has_online_backup')
sns.boxplot(x="has_online_backup", y="churn", data=train)

# has_device_protect
plt.subplot(428)
plt.title('churn x has_device_protect')
sns.boxplot(x="has_device_protect", y="churn", data=train)

# is_streaming_tv
plt.subplot(422)
plt.title('churn x is_streaming_tv')
sns.boxplot(x="is_streaming_tv", y="churn", data=train)

# is_streaming_movies
plt.subplot(424)
plt.title('churn x is_streaming_movies')
sns.boxplot(x="is_streaming_movies", y="churn", data=train)

# is_male
plt.subplot(426)
plt.title('churn x is_male')
sns.boxplot(x="is_male", y="churn", data=train)

# has_partner
plt.subplot(428)
plt.title('churn x has_partner')
sns.boxplot(x="has_partner", y="churn", data=train)

# has_dependents
plt.subplot(422)
plt.title('churn x has_dependents')
sns.boxplot(x="has_dependents", y="churn", data=train)

# has_phone_serv
plt.subplot(424)
plt.title('churn x has_phone_serv')
sns.boxplot(x="has_phone_serv", y="churn", data=train)

# is_male
plt.subplot(426)
plt.title('churn x is_male')
sns.boxplot(x="is_male", y="churn", data=train)

# has_paperless_bill
plt.subplot(428)
plt.title('churn x has_paperless_bill')
sns.boxplot(x="has_paperless_bill", y="churn", data=train)

# bank_transfer
plt.subplot(428)
plt.title('churn x bank_transfer')
sns.boxplot(x="bank_transfer", y="churn", data=train)

# churn x has_tech_support
plt.subplot(423)
plt.title('churn x has_tech_support')
sns.barplot(x="has_tech_support", y="churn", data=train)
plt.legend
print('has_tech_support :' f'\n{train.has_tech_support.describe()}\n')
# churn x has_online_backup
plt.subplot(424)
plt.title('churn x has_online_backup')
sns.barplot(x="has_online_backup", y="churn", data=train)
plt.legend
print('has_online_backup:' f'\n{train.has_online_backup.describe()}\n')
# churn x has_device_protect
plt.subplot(425)
plt.title('churn x has_device_protect')
sns.barplot(x="has_device_protect", y="churn", data=train)
plt.legend
print('has_device_protect:' f'\n{train.has_device_protect.describe()}\n')
plt.figure(figsize=(15,15))
# churn x is_streaming_tv
plt.subplot(421)
plt.title('churn x is_streaming_tv')
sns.barplot(x="is_streaming_tv", y="churn", data=train)
plt.legend
print('is_streaming_tv:' f'\n{train.is_streaming_tv.describe()}\n')
# churn x is_streaming_movies 
plt.subplot(422)
plt.title('churn x is_streaming_movies')
sns.barplot(x="is_streaming_movies", y="churn", data=train)
plt.legend
print('is_streaming_movies :' f'\n{train.is_streaming_movies.describe()}\n')

plt.figure(figsize=(15,15))
# churn x has_paperless_bill
plt.subplot(421)
plt.title('churn x has_paperless_bill')
sns.barplot(x="has_paperless_bill", y="churn", data=train)
plt.legend
print('has_paperless_bill:' f'\n{train.has_paperless_bill.describe()}\n')
# churn x bank_transfer
plt.subplot(422)
plt.title('churn x bank_transfer')
sns.barplot(x="bank_transfer", y="churn", data=train)
plt.legend
print('bank_transfer:' f'\n{train.bank_transfer.describe()}\n')
# churn x credit_card
plt.subplot(423)
plt.title('churn x credit_card')
sns.barplot(x="credit_card", y="churn", data=train)
plt.legend
print('credit_card:' f'\n{train.credit_card.describe()}\n')
# churn x e_check 
plt.subplot(424)
plt.title('churn x e_check')
sns.barplot(x="e_check", y="churn", data=train)
plt.legend
print('e_check :' f'\n{train.e_check.describe()}\n')
# churn x mailed_check
plt.subplot(425)
plt.title('churn x mailed_check')
sns.barplot(x="mailed_check", y="churn", data=train)
plt.legend
print('mailed_check:' f'\n{train.mailed_check.describe()}\n')
# churn x m_to_m_contract
plt.subplot(423)
plt.title('churn x m_to_m_contract')
sns.barplot(x="m_to_m_contract", y="churn", data=train)
plt.legend
print('m_to_m_contract:' f'\n{train.m_to_m_contract.describe()}\n')
# churn x one_yr_contract
plt.subplot(424)
plt.title('churn x one_yr_contract')
sns.barplot(x="one_yr_contract", y="churn", data=train)
plt.legend
print('one_yr_contract:' f'\n{train.one_yr_contract.describe()}\n')
# churn x two_yr_contract
plt.subplot(425)
plt.title('churn x two_yr_contract')
sns.barplot(x="two_yr_contract", y="churn", data=train)
plt.legend
print('two_yr_contract:' f'\n{train.two_yr_contract.describe()}\n')
plt.figure(figsize=(15,15))
# churn x has_phone_serv 
plt.subplot(421)
plt.title('churn x has_phone_serv')
sns.barplot(x="has_phone_serv", y="churn", data=train)
plt.legend
print('has_phone_serv :' f'\n{train.has_phone_serv.describe()}\n')
# churn x has_online_secur
plt.subplot(422)
plt.title('churn x has_online_secur')
sns.barplot(x="has_online_secur", y="churn", data=train)
plt.legend
print('has_online_secur:' f'\n{train.has_online_secur.describe()}\n')
plt.tight_layout()
plt.tight_layout()plt.tight_layout()x

# churn x tenure
plt.subplot(423)
plt.title('churn x tenure')
sns.violinplot(x="tenure", y="churn", data=train)
plt.subplot(424)
plt.title('churn x tenure')
sns.boxplot(x="tenure", y="churn", data=train)
plt.legend
print('Tenure:' f'\n{train.tenure.describe()}\n')
# churn x monthly_charges
plt.subplot(421)
plt.title('churn x monthly_charges')
sns.boxplot(x="monthly_charges", y="churn", data=train)
plt.subplot(422)
plt.title('churn x monthly_charges')
sns.violinplot(x="monthly_charges", y="churn", data=train)
plt.legend
print('monthly_charges:' f'\n{train.monthly_charges.describe()}\n')
# churn x total_charges
plt.subplot(423)
plt.title('churn x total_charges')
sns.violinplot(x="total_charges", y="churn", data=train)
plt.legend
print('total_charges:' f'\n{train.total_charges.describe()}\n')
# churn x is_male
plt.subplot(424)
plt.title('churn x is_male')
sns.barplot(x="is_male", y="churn", data=train)
plt.legend
print('is_male:' f'\n{train.is_male.describe()}\n')
# churn x senior_citizen
plt.subplot(421)
plt.title('churn x senior_citizen')
sns.barplot(x="senior_citizen", y="churn", data=train)
plt.legend
print('Senior citizen:' f'\n{train.senior_citizen.describe()}\n')


SyntaxError: invalid syntax (4149801365.py, line 183)

In [ ]:
train['total_charges'] = train['total_charges'].replace(" ", np.nan)
train = train[train["total_charges"].notnull()]
train = train.reset_index()[train.columns]
train['total_charges'] = pd.to_numeric(train['total_charges'])

train.dtypes

In [21]:
train.has_online_secur.value_counts()

0    2795
1    1148
Name: has_online_secur, dtype: int64